In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

from sklearn.tree import DecisionTreeClassifier

In [2]:
data = torch.load('spec_datasets_4.pt')

C:\Users\user\AppData\Local\Temp\ipykernel_6440\1509188321.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('spec_datasets_4.pt')


In [3]:
# 학습, 검증, 테스트 데이터 분할
X_train, X_test, y_train, y_test = data['X_train'], data['X_test'], data['y_train'], data['y_test']
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_dev, y_dev = data['X_dev'], data['y_dev']

In [4]:
X_train = X_train.cpu().numpy()  # PyTorch 텐서를 NumPy 배열로 변환
X_val = X_val.cpu().numpy()
y_train = y_train.cpu().numpy()
y_val = y_val.cpu().numpy()

X_test = X_test.cpu().numpy()
y_test = y_test.cpu().numpy()

X_dev = X_dev.cpu().numpy()
y_dev = y_dev.cpu().numpy()

In [5]:
X_train = np.squeeze(X_train, axis=1)
X_val = np.squeeze(X_val, axis=1)

X_test = np.squeeze(X_test, axis=1)

X_dev = np.squeeze(X_dev, axis=1)

In [6]:
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(36, 2048)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu')
])

cnn_model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

cnn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# CNN 모델로 특징 추출
cnn_features = cnn_model.predict(X_train)

# Decision Tree 모델로 학습
decision_tree = DecisionTreeClassifier()
decision_tree.fit(cnn_features, y_train)



Epoch 1/50
65/65 [==============================] - 2s 19ms/step - loss: 1.5898 - accuracy: 0.0077 - val_loss: 1.1867 - val_accuracy: 0.1019
Epoch 2/50
65/65 [==============================] - 1s 17ms/step - loss: 1.1791 - accuracy: 0.1144 - val_loss: 1.1559 - val_accuracy: 0.1288
Epoch 3/50
65/65 [==============================] - 1s 18ms/step - loss: 1.1366 - accuracy: 0.1274 - val_loss: 1.0989 - val_accuracy: 0.1615
Epoch 4/50
65/65 [==============================] - 1s 17ms/step - loss: 1.0343 - accuracy: 0.1639 - val_loss: 0.9449 - val_accuracy: 0.2000
Epoch 5/50
65/65 [==============================] - 1s 18ms/step - loss: 0.8461 - accuracy: 0.2192 - val_loss: 0.7506 - val_accuracy: 0.2500
Epoch 6/50
65/65 [==============================] - 1s 17ms/step - loss: 0.5895 - accuracy: 0.2505 - val_loss: 0.4912 - val_accuracy: 0.2346
Epoch 7/50
65/65 [==============================] - 1s 18ms/step - loss: 0.4178 - accuracy: 0.2529 - val_loss: 0.2746 - val_accuracy: 0.2346
Epoch 8/50
65

DecisionTreeClassifier()

In [9]:
# 모델 평가
cnn_test_features = cnn_model.predict(X_test)
y_pred = decision_tree.predict(cnn_test_features)
# 모델 예측
print(y_pred, y_test)

1/1 [==============================] - 0s 14ms/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]


In [10]:
from sklearn.metrics import classification_report

# 테스트 데이터에 대한 예측
# y_pred_classes = (y_pred > 0.6).astype(int)  # 0.8 이상의 확률을 정상(1)으로 변환

# 정확도, 정밀도, 재현율, F1 스코어 계산 및 출력
print(classification_report(y_test, y_pred, target_names=["비정상", "정상"]))

              precision    recall  f1-score   support

         비정상       0.00      0.00      0.00         2
          정상       0.90      1.00      0.95        18

    accuracy                           0.90        20
   macro avg       0.45      0.50      0.47        20
weighted avg       0.81      0.90      0.85        20



C:\Users\user\anaconda3\envs\pdm\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\pdm\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\pdm\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
